In [111]:
raw_data_dir = '../data/raw/'
interim_data_dir = '../data/interim'

In [150]:
import requests
import json
import pandas as pd

pd.set_option('display.max_columns', 500)

url= "https://crashviewer.nhtsa.dot.gov/CrashAPI"
#/crashes/GetCrashesByLocation?fromCaseYear=2014&toCaseYear=2015&state=1&county=1&format=json

fromCaseYear = "2010"
toCaseYear = "2020"
state = "6"
qurl = f"{url}/crashes/GetCrashesByLocation?fromCaseYear={fromCaseYear}&toCaseYear={toCaseYear}&state={state}&county=73&format=json"

cali=requests.get(qurl).json()
data = (json.dumps(cali,indent=4,sort_keys=True))

CITYNAME = []
COUNTY =  []
COUNTYNAME = []
CaseYear = []
FATALS = []
LATITUDE = []
LONGITUD = []
STATE = []
STATENAME = []
ST_CASE = []
TOTALVEHICLES = []
TWAY_ID = []
TWAY_ID2 = []
VE_FORMS = []
# Loop through the list of cities and perform a request for data on each
for COUNTYNAME in COUNTYNAME:
    response = requests.get(qurl).json()
    CITYNAME.append(response[0]['CITYNAME'])


In [146]:
df = pd.DataFrame(cali['Results'][0])
# for i, r in df.iterrows():
#     print(i)
#     print(r["CaseYear"])
#     year = r["CaseYear"]
df[df['ST_CASE'] == '63474']

,CITY,CITYNAME,COUNTY,COUNTYNAME,CaseYear,FATALS,LATITUDE,LONGITUD,STATE,STATENAME,ST_CASE,TOTALVEHICLES,TWAY_ID,TWAY_ID2,VE_FORMS
1890,710,CHULA VISTA,73,SAN DIEGO (73),2016,1,32.60915556,-117.019969440,6,California,63474,2,OLYMPIC PKWY,,2


In [154]:
fromCaseYear = "2010"
toCaseYear = "2020"
state = "6"

qurl = f"{url}/crashes/GetCaseDetails?stateCase=63474&caseYear=2016&state=6&format=json"
data = requests.get(qurl).json()
case = data['Results'][0][0]['CrashResultSet']
case['Vehicles'][1]['Persons'][0]

{'AGE': '22',
 'AGENAME': '22 Years',
 'AIR_BAG': '9',
 'AIR_BAGNAME': 'Deployment- Unknown Location',
 'ALC_DET': '1',
 'ALC_DETNAME': 'Evidential Test (breath, blood, urine)',
 'ALC_RES': '0',
 'ALC_RESNAME': '0.000 % BAC',
 'ALC_STATUS': '2',
 'ALC_STATUSNAME': 'Test Given',
 'ATST_TYP': '1',
 'ATST_TYPNAME': 'Blood',
 'BODY_TYP': '4',
 'BODY_TYPNAME': '4-door sedan, hardtop',
 'CERT_NO': None,
 'COUNTY': '73',
 'COUNTYNAME': 'SAN DIEGO (73)',
 'CaseYear': '2016',
 'DAY': '25',
 'DEATH_DA': '26',
 'DEATH_DANAME': '26',
 'DEATH_HR': '14',
 'DEATH_HRNAME': '14:00-14:59',
 'DEATH_MN': '38',
 'DEATH_MNNAME': '38',
 'DEATH_MO': '11',
 'DEATH_MONAME': 'November',
 'DEATH_TM': '1438',
 'DEATH_YR': '2016',
 'DEATH_YRNAME': '2016',
 'DOA': '0',
 'DOANAME': 'Not Applicable',
 'DRINKING': '0',
 'DRINKINGNAME': 'No (Alcohol Not Involved)',
 'DRUGRES1': '600',
 'DRUGRES1NAME': 'DELTA 9',
 'DRUGRES2': '605',
 'DRUGRES2NAME': 'Tetrahydrocannabinols (THC)',
 'DRUGRES3': '0',
 'DRUGRES3NAME': 'Test 

In [163]:
def extract_people(v):
    for p in v['Persons']:
        yield {
            'Speed Limit Exceeded': v['SPEEDRELNAME'],
            'Speed limit': v['TRAV_SP'],
            'Vin Number': v['VINNAME'],
            'Traveled Speed Veh': v['VSPD_LIM'],
            'Make': v['MAKENAME'],
            'Make/Model': v['MAK_MODNAME'],
            'Model': v['MODELNAME'],
            'Type of Vehicle': v['BODY_TYPNAME'],
            
            "Age": p['AGE'],
            "Age Name": p['AGENAME'],
            "County": p['COUNTYNAME'],
            "Death Day of Month": p['DEATH_DANAME'],
            "DOA Name": p['DOANAME'],
            # injury sev
            "Injury Severity Name": p['INJ_SEVNAME'],
            "Race": p['RACENAME'],
            "Road Type": p["ROAD_FNCNAME"],
            "Sex": p["SEXNAME"],
            "Make": p["MAKENAME"],
        }


        
def get_people(case):

    hour = case['HOUR']
    minute = case['MINUTE']
    time = f"{hour}:{minute}"
        
    accident_info = {
        'Case Number': case['ST_CASE'],
        "Description of Veh Coll": case['CF2NAME'], 
        "Day of Week": case['DAY_WEEKNAME'],
        "Drunk Driver": case['DRUNK_DR'],
        "Year": case['CaseYear'],
        "Month": case['MonthName'],
        "Hour": hour,
        "Time of Accident": time,
    }

    vehicles = case['Vehicles']
    
    people = [{**accident_info, **p} for v in vehicles for p in extract_people(v)]
    
    return pd.DataFrame(people)


def get_events(case):
    c_events = [{
        'Case Number': case['ST_CASE'],
# In a traffic accident AOI is Area of Impact. The spot the two cars collided is measured 
# to a fixed object, usually the curb, so it can be reconstructed later.
        'Area of Impact': e['AOI1NAME'],
# standard of evidence
# https://safety.fhwa.dot.gov/rsdp/cdip_rpti.aspx
        'Standard of Evenidence': e['SOENAME'],
        'Event Number': e['EVENTNUM'],
        'Vehicle 1': e['VNUMBER1'],
        'Vehicle 2': e['VNUMBER2'],
    } for e in case['CEvents']]
    
    return pd.DataFrame(c_events)




In [164]:
get_people(case)

,Age,Age Name,Case Number,County,DOA Name,Day of Week,Death Day of Month,Description of Veh Coll,Drunk Driver,Hour,Injury Severity Name,Make,Make/Model,Model,Month,Race,Road Type,Sex,Speed Limit Exceeded,Speed limit,Time of Accident,Traveled Speed Veh,Type of Vehicle,Vin Number,Year
0,40,40 Years,63474,SAN DIEGO (73),Not Applicable,Friday,Not Applicable (Non-Fatal),None,0,11,Possible Injury (C),Honda,Honda CR-V,CR-V,November,Not a Fatality (not Applicable),None,Male,No,998,11:51,50,"Compact Utility (Utility Vehicle Categories ""S...",2HKRM4H79EH7,2016
1,36,36 Years,63474,SAN DIEGO (73),Not Applicable,Friday,Not Applicable (Non-Fatal),None,0,11,No Apparent Injury (O),Honda,Honda CR-V,CR-V,November,Not a Fatality (not Applicable),None,Female,No,998,11:51,50,"Compact Utility (Utility Vehicle Categories ""S...",2HKRM4H79EH7,2016
2,11,11 Years,63474,SAN DIEGO (73),Not Applicable,Friday,Not Applicable (Non-Fatal),None,0,11,Possible Injury (C),Honda,Honda CR-V,CR-V,November,Not a Fatality (not Applicable),None,Female,No,998,11:51,50,"Compact Utility (Utility Vehicle Categories ""S...",2HKRM4H79EH7,2016
3,22,22 Years,63474,SAN DIEGO (73),Not Applicable,Friday,26,None,0,11,Fatal Injury (K),BMW,BMW 3-series,3-series,November,Black,None,Male,"Yes, Too Fast for Conditions",77,11:51,50,"4-door sedan, hardtop",WBAAM3340YKC,2016
4,19,19 Years,63474,SAN DIEGO (73),Not Applicable,Friday,Not Applicable (Non-Fatal),None,0,11,Suspected Minor Injury (B),BMW,BMW 3-series,3-series,November,Not a Fatality (not Applicable),None,Male,"Yes, Too Fast for Conditions",77,11:51,50,"4-door sedan, hardtop",WBAAM3340YKC,2016


In [152]:
get_events(case)

,Area of Impact,Case Number,Event Number,Standard of Evenidence,Vehicle 1,Vehicle 2
0,Right,63474,1,Motor Vehicle In-Transport,1,2
1,Non-Harmful Event,63474,2,Ran Off Roadway - Right,1,5555
2,Not Reported,63474,3,Curb,1,9999
3,Not Reported,63474,4,Tree (Standing Only),1,9999


In [ ]:
# Get average traffic volume for zip code possibly of google maps